### Establish a connection

In [7]:
from omero.gateway import BlitzGateway, MapAnnotationWrapper
from PIL import Image
import matplotlib.pyplot as plt
import cv2

conn = BlitzGateway("USERNAME", "PASSWORD", host="wsi.lavlab.mcw.edu", secure=True)
conn.connect()
conn.c.enableKeepAlive(60)

### Get all tiles from each image in a folder
            

In [ ]:
import importlib
import ipynb.fs.full.functions
importlib.reload(ipynb.fs.full.functions)
from ipynb.fs.full.functions import getTilesFromImage, getUnlabeledTilesFromImage, getEdgeTilesFromImage

folderIds = [566, 568, 567, 309, 553, 652, 571, 569, 651, 570]

for folderId in folderIds:
    print(f"processing folder {folderId}")
    # get all images from a folder
    dataset = conn.getObject("Dataset", folderId)
    datasetImages = dataset.listChildren()

    # loop through each image in the folder individually
    for image in datasetImages:
        imageId = image.getId()
        print(f"processing image {imageId}")
        allBoxes = []

        # get tumorous tiles
        print("getting tumorous tiles")
        tiles, boxes1 = getTilesFromImage(imageId, conn)
        allBoxes.extend(boxes1)

        # tiles from the edge (background)
        print("getting background tiles")
        backgroundTiles, boxes2 = getEdgeTilesFromImage(imageId, conn, 500, 500)
        allBoxes.extend(boxes2)

        # get unlabeled tiles
        print("getting unlabeled tiles")
        unlabeledTiles = getUnlabeledTilesFromImage(imageId, conn, 500, 500, allBoxes)

        # save tumorous tiles
        print("saving tumorous tiles")
        index = 0
        for tile in tiles:
            filename = f"./training/{tile['gleason']}/img{tile['imgId']}roi{tile['roiId']}id{index}.tiff"
            cv2.imwrite(filename, cv2.cvtColor(tile["tile"], cv2.COLOR_RGB2BGR))
            index += 1

        # save background tiles
        print("saving background tiles")
        index = 0
        for tile in backgroundTiles:
            filename = f"./training/Background/img{tile['imgId']}id{index}.tiff"
            cv2.imwrite(filename, cv2.cvtColor(tile["tile"], cv2.COLOR_RGB2BGR))
            index += 1
        
        # save unlabeled tiles
        print("saving unlabeled tiles")
        index = 0
        for tile in unlabeledTiles:
            filename = f"./training/Unlabeled/img{tile['imgId']}id{index}.tiff"
            cv2.imwrite(filename, cv2.cvtColor(tile["tile"], cv2.COLOR_RGB2BGR))
            index += 1